In [1]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [3]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 1

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 13, 13, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
Softmax (Dense)              (None, 10)                13530     
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 0s 2ms/step - loss: 0.4322 - accuracy: 0.8489

Test accuracy: 0.8489000201225281


In [5]:
import os

MODEL_DIR = os.getcwd()+""
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)


export_path = /Users/ramteja/Downloads/deploy_tensor/1

INFO:tensorflow:Assets written to: /Users/ramteja/Downloads/deploy_tensor/1/assets



export_path = /Users/ramteja/Downloads/deploy_tensor/1

running these comands in the start docker with tenserflow serving

$ docker pull tensorflow/serving

-----> problem is at this command i am not able figurout bro it shows it is not able to fine the path. 

try one:
$ docker run -p 8501:8501 --mount type=bind,source=/Users/ramteja/Downloads/deploy_tensor/1,target=/models/1 -e MODEL_NAME=test_model -t tensorflow/serving

Error:
2020-10-13 17:41:14.543070: I tensorflow_serving/model_servers/server.cc:87] Building single TensorFlow model file config:  model_name: test_model model_base_path: /models/test_model
2020-10-13 17:41:14.543342: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2020-10-13 17:41:14.543903: I tensorflow_serving/model_servers/server_core.cc:575]  (Re-)adding model: test_model
2020-10-13 17:41:14.547615: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:362] FileSystemStoragePathSource encountered a filesystem access error: Could not find base path /models/test_model for servable test_model
2020-10-13 17:41:15.551822: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:362] FileSystemStoragePathSource encountered a filesystem access error: Could not find base path /models/test_model for servable test_model




Try two:
$ docker run -p 8501:8501 --mount type=bind,source=/Users/ramteja/Downloads/deploy_tensor/1,target=/models/test_model -e MODEL_NAME=test_model -t tensorflow/serving

Error:

2020-10-13 17:39:57.973846: I tensorflow_serving/model_servers/server.cc:87] Building single TensorFlow model file config:  model_name: test_model model_base_path: /models/test_model
2020-10-13 17:39:57.978167: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2020-10-13 17:39:57.978578: I tensorflow_serving/model_servers/server_core.cc:575]  (Re-)adding model: test_model
2020-10-13 17:39:57.997829: W tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:267] No versions of servable test_model found under base path /models/test_model




Extra info:
i tryed default as show in this every thing works 

# Download the TensorFlow Serving Docker image and repo
docker pull tensorflow/serving

git clone https://github.com/tensorflow/serving
# Location of demo models
TESTDATA="$(pwd)/serving/tensorflow_serving/servables/tensorflow/testdata"

# Start TensorFlow Serving container and open the REST API port
docker run -t --rm -p 8501:8501 \
    -v "$TESTDATA/saved_model_half_plus_two_cpu:/models/half_plus_two" \
    -e MODEL_NAME=half_plus_two \
    tensorflow/serving &

# Query the model using the predict API
curl -d '{"instances": [1.0, 2.0, 5.0]}' \
    -X POST http://localhost:8501/v1/models/half_plus_two:predict

# Returns => { "predictions": [2.5, 3.0, 4.5] }
